In [41]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [51]:
def parse(url):
  # url = "http://news.bbc.co.uk/2/hi/health/2284783.stm"
  html = urlopen(url).read()
  soup = BeautifulSoup(html, features="html.parser")

  # kill all script and style elements
  for script in soup(["script", "style"]):
      script.extract()    # rip it out

  # get text
  text = soup.get_text()

  # break into lines and remove leading and trailing space on each
  lines = (line.strip() for line in text.splitlines())
  # break multi-headlines into a line each
  chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
  # drop blank lines
  text = '\n'.join(chunk for chunk in chunks if chunk)

  return text

In [44]:
# text1

'BBC NEWS | Health | Blondes \'to die out in 200 years\'\nNEWS\nSPORT\nWEATHER\nWORLD SERVICE\nA-Z INDEX\nSEARCH\nYou are in:\xa0Health\nNews Front Page\nAfrica\nAmericas\nAsia-Pacific\nEurope\nMiddle East\nSouth Asia\nUK\nBusiness\nEntertainment\nScience/Nature\nTechnology\nHealth\nMedical notes\n-------------\nTalking Point\n-------------\nCountry Profiles\nIn Depth\n-------------\nProgrammes\n-------------\nSERVICES\nDaily E-mail\nNews Ticker\nMobile/PDAs\n-------------\nText Only\nFeedback\nHelp\nEDITIONS\nChange to UK\nFriday, 27 September, 2002, 11:51 GMT 12:51 UK\nBlondes \'to die out in 200 years\'\nScientists believe the last blondes will be in Finland\nThe last natural blondes will die out within 200 years, scientists believe.\nA study by experts in Germany suggests people with blonde hair are an endangered species and will become extinct by 2202.\nResearchers predict the last truly natural blonde will be born in Finland - the country with the highest proportion of blondes.\n

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [8]:
data_fake = pd.read_csv('Fake.csv')
data_true = pd.read_csv('True.csv')
data_fake.head()
data_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [9]:
data_fake["class"] = 0
data_true["class"] = 1

In [10]:
data_fake.shape
data_true.shape

(21417, 5)

In [11]:
data_fake_manual_testing = data_fake.tail(10)
for i in range(23480, 23470, -1):
  data_fake.drop([i], axis = 0, inplace = True)

data_true_manual_testing = data_true.tail(10)
for i in range(21416, 21406, -1):
  data_true.drop([i], axis = 0, inplace = True)

In [12]:
data_fake_manual_testing["class"] = 0
data_true_manual_testing["class"] = 1

<ipython-input-12-a26436eb6887>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_manual_testing["class"] = 0
<ipython-input-12-a26436eb6887>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_manual_testing["class"] = 1


In [13]:
data_merge = pd.concat([data_fake, data_true], axis = 0)
data_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [14]:
data = data_merge.drop(['title','subject','date'], axis = 1)

In [15]:
def wordopt(text):
  text = text.lower()
  text = re.sub('\[.*?\]', '', text)
  text = re.sub("\\W", " ", text)
  text = re.sub('https?://\S+|www\.\S+', '', text)
  text = re.sub('<.*?>+', '', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  text = re.sub('\n', '', text)
  text = re.sub('\w*\d\w', '', text)
  return text

In [16]:
wordopt(text)

'bbc news   health   blondes  to die out in  years  news sport weather world service a z index search you are in  health news front page africa americas asia pacific europe middle east south asia uk business entertainment science nature technology health medical notes               talking point               country profiles in depth               programmes               services daily e mail news ticker mobile pdas               text only feedback help editions change to uk friday   september      gmt   uk blondes  to die out in  years  scientists believe the last blondes will be in finland the last natural blondes will die out within  years  scientists believe  a study by experts in germany suggests people with blonde hair are an endangered species and will become extinct by   researchers predict the last truly natural blonde will be born in finland   the country with the highest proportion of blondes  the frequency of blondes may drop but they won t disappear prof jonathan rees  u

In [17]:
data['text'] = data['text'].apply(wordopt)

In [18]:
x = data['text']
y = data['class']

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [23]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [24]:
pred_lr = LR.predict(xv_test)
LR.score(xv_test, y_test)
classification_report(y_test, pred_lr)

'              precision    recall  f1-score   support\n\n           0       0.99      0.99      0.99      5865\n           1       0.98      0.99      0.99      5355\n\n    accuracy                           0.99     11220\n   macro avg       0.99      0.99      0.99     11220\nweighted avg       0.99      0.99      0.99     11220\n'

In [25]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [26]:
pred_dt = DT.predict(xv_test)
DT.score(xv_test, y_test)
classification_report(y_test, pred_dt)

'              precision    recall  f1-score   support\n\n           0       0.99      1.00      1.00      5865\n           1       1.00      0.99      0.99      5355\n\n    accuracy                           0.99     11220\n   macro avg       0.99      0.99      0.99     11220\nweighted avg       0.99      0.99      0.99     11220\n'

In [32]:
def output_lable(n):
  if n == 0:
    return "Fake News"
  elif n == 1:
    return "True News"

In [55]:
def manual_testing(news):
  testing_news = {"text":[news]}
  new_def_test = pd.DataFrame(testing_news)
  new_def_test["text"] = new_def_test["text"].apply(wordopt)
  new_x_test = new_def_test["text"]
  new_xv_test = vectorization.transform(new_x_test)
  pred_LR = LR.predict(new_xv_test)
  # pred_DT = DT.predict(new_xv_test)

  return print("\n\nLR Prediction: {}".format(output_lable(pred_LR[0])))

In [50]:
# input a news article text

news = str(input())
manual_testing(news)

After wrapping up a three-day-visit to the United States, Prime Minister Narendra Modi on Saturday, 24 June, began his official two-day visit to Egypt's Cairo.  This is the first bilateral visit to Egypt by an Indian PM since 1997.  During this visit, Modi will hold strategic talks with President Abdel Fattah El-Sisi. In a statement on 20 June, PM Modi said:  "We had the pleasure of receiving President Sisi as the Chief Guest at our Republic Day celebrations this year. These two visits in the span of a few months are a reflection of our rapidly evolving partnership with Egypt, which was elevated to a 'Strategic Partnership' during President Sisi's visit." After his arrival, PM Modi was personally received by Egyptian Prime Minister Mostafa Madbouly at the Cairo airport.


LR Prediction: True News 
DT Prediction Fake News


In [57]:
# OR input a news article url

news = str(input())
text = parse(news)
manual_testing(text)

https://www.ndtv.com/india-news/manipur-manipur-violence-surrounded-by-mob-of-1-200-in-manipur-village-army-releases-12-militants-4150479


LR Prediction: True News
